In [10]:
!pip install datasets gitpython

# ------------------------------------------------
# Imports
# ------------------------------------------------
import os
import subprocess
from datasets import load_dataset
import git

# ------------------------------------------------
# Load SWE-bench Lite dataset
# ------------------------------------------------
dataset = load_dataset("princeton-nlp/SWE-bench_Lite", split="test")

# ------------------------------------------------
# Utility: Analyze an instance
# ------------------------------------------------
def analyze_instance(instance):
    """Analyze a SWE-bench instance to understand the problem."""
    print("="*60)
    print(f"REPOSITORY:   {instance['repo']}")
    print(f"INSTANCE ID:  {instance['instance_id']}")
    print("="*60)
    
    print("PROBLEM STATEMENT:")
    print(instance["problem_statement"])
    print()
    
    if "hints_text" in instance and instance["hints_text"]:
        print("HINTS:")
        print(instance["hints_text"])
        print()
    
    return instance

# Example: Inspect first instance
instance = analyze_instance(dataset[0])

# ------------------------------------------------
# Utility: Setup repository at base commit
# ------------------------------------------------
def setup_repository(instance, base_dir="/content/repos"):
    """
    Clone and checkout the repository at the base commit.
    Also tries to install repo dependencies automatically.
    """
    repo_name = instance["repo"]
    base_commit = instance["base_commit"]
    
    repo_path = os.path.join(base_dir, repo_name.replace("/", "__"))
    os.makedirs(base_dir, exist_ok=True)
    
    # Clone repo if not already cloned
    if not os.path.exists(repo_path):
        print(f"Cloning {repo_name} into {repo_path}...")
        git.Repo.clone_from(f"https://github.com/{repo_name}.git", repo_path)
    
    # Checkout base commit
    repo = git.Repo(repo_path)
    repo.git.checkout(base_commit)
    
    print(f"✅ Repository {repo_name} is set up at commit {base_commit}")
    
    # Try installing dependencies
    if os.path.exists(os.path.join(repo_path, "requirements.txt")):
        print("📦 Installing dependencies from requirements.txt...")
        subprocess.run(["pip", "install", "-r", "requirements.txt"], cwd=repo_path)
    
    elif os.path.exists(os.path.join(repo_path, "setup.py")):
        print("📦 Installing repo via setup.py...")
        subprocess.run(["pip", "install", "-e", "."], cwd=repo_path)
    
    elif os.path.exists(os.path.join(repo_path, "pyproject.toml")):
        print("📦 Installing repo via pyproject.toml...")
        subprocess.run(["pip", "install", "."], cwd=repo_path)
    
    else:
        print("⚠️ No dependency file found. You may need to install manually.")
    
    return repo_path

# Example: Setup repo for first instance
repo_path = setup_repository(instance)

# ------------------------------------------------
# Generate solution (stub — replace with LLM call)
# ------------------------------------------------
def generate_solution(instance, model_api=None):
    """
    Generate a solution using a preferred LLM API.
    NOTE: This is pseudocode. Replace with your actual model API call.
    """
    prompt = f"""
    You are a software engineer working on the repository {instance['repo']}.

    PROBLEM:
    {instance['problem_statement']}

    Please analyze the codebase and provide a patch that resolves this issue.
    Your response should include:
    1. Analysis of the problem
    2. Files that need to be modified
    3. Specific code changes

    Format your response as a unified diff patch.
    """
    
    # TODO: Replace this with a real API call
    dummy_patch = """diff --git a/example.py b/example.py
index e69de29..4b825dc 100644
--- a/example.py
+++ b/example.py
@@ -0,0 +1,2 @@
+def hello():
+    return "Hello SWE-bench"
"""
    return dummy_patch

# Example: Generate dummy solution
patch = generate_solution(instance)
print(patch)

# ------------------------------------------------
# Evaluate solution with pytest
# ------------------------------------------------
def evaluate_solution(instance, patch, repo_path):
    """
    Apply the patch and run tests using pytest.
    Args:
        instance: SWE-bench instance.
        patch: Unified diff patch string.
        repo_path: Path to the repo.
    """
    patch_file = os.path.join(repo_path, "solution.patch")
    
    # Write patch
    with open(patch_file, "w") as f:
        f.write(patch)
    
    try:
        # Apply patch
        subprocess.run(["git", "apply", patch_file], cwd=repo_path, check=True)
        
        # Run tests
        print("🧪 Running pytest...")
        result = subprocess.run(["pytest", "-q"], cwd=repo_path, 
                                capture_output=True, text=True)
        
        if result.returncode == 0:
            print("✅ Solution appears to work!")
            return True
        else:
            print("❌ Tests failed:")
            print(result.stdout)
            return False
            
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to apply patch: {e}")
        return False

# Example: Evaluate dummy solution
evaluate_solution(instance, patch, repo_path)


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
REPOSITORY:   astropy/astropy
INSTANCE ID:  astropy__astropy-12907
PROBLEM STATEMENT:
Modeling's `separability_matrix` does not compute separability correctly for nested CompoundModels
Consider the following model:

```python
from astropy.modeling import models as m
from astropy.modeling.separable import separability_matrix

cm = m.Linear1D(10) & m.Linear1D(5)
```

It's separability matrix as you might expect is a diagonal:

```python
>>> separability_matrix(cm)
array([[ True, False],
       [False,  True]])
```

If I make the model more complex:
```python
>>> separability_matrix(m.Pix2Sky_TAN() & m.Linear1D(10) & m.Linear1D(5))
array([[ True,  True, False, False],
       [ True,  True, False, False],
       [False, False,  True, False],
       [False, False, False,  True]])
```

The output matrix is again, as expected, the outputs and inputs to the linear models are separable and independent of each other.



False